In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
#print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [11]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os
import pandas as pd
import numpy as np
import random
random.seed(7)
import time

In [12]:
then = time.time()
df = pd.read_csv('data/train/train.csv',dtype = {'acoustic_data': np.int16, 'time_to_failure': np.float64} ) # float32 is enough :)
now = time.time()
print(now-then)

129.52645611763


In [16]:
ttf = df['time_to_failure'].values
index_start = np.nonzero(np.diff(ttf) > 0)[0] + 1
index_start = np.insert(index_start, 0, 0)
dict_df={}
for i in range(len(index_start)):
    if i<(len(index_start)-1):
        df_tmp=df[index_start[i]:index_start[i+1]]
    else:
        df_tmp=df[index_start[i]:]       
    dict_df["df"+str(i)]=df_tmp

In [21]:
len(dict_df)

17

In [22]:
def randomChoice(l):
    return random.randint(0, l - 1)

150000

In [87]:
def randomTrainingExample(df_dict):
    k = len(df_dict)
    num=randomChoice(k)
    #print(num)
    len_df=len(df_dict['df'+str(num)])
    idx_start=random.randint(0,len_df-149999)
    idx_end=idx_start+150000
    #print('Start Index:',idx_start)
    #print('End Index:',idx_end)
    df_tmp=df_dict['df'+str(num)]
    sample_x=df_tmp.iloc[idx_start:idx_end]['acoustic_data']
    sample_y=df_tmp.iloc[idx_start:idx_end]['time_to_failure']

    sample_x1=np.diff(sample_x)
    sample_y1=np.diff(sample_y)
    meanx1=np.mean(sample_x1)
    meany1=np.mean(sample_y1)
    sample_x1=np.append(sample_x1,meanx1)
    sample_y1=np.append( sample_y1,meany1)

    sample_x2=np.diff(sample_x1)
    sample_y2=np.diff(sample_y1)
    meanx2=np.mean(sample_x2)
    meany2=np.mean(sample_y2)
    sample_x2=np.append(sample_x2,meanx2)
    sample_y2=np.append(sample_y2,meany2)
  #   sample_y2=np.append(sample_y2,meany2)

    sample_x=np.array(sample_x)
    sample_y=np.array(sample_y)
    sample_x1=np.array(sample_x1)
    sample_y1=np.array(sample_y1)
    sample_x2=np.array(sample_x2)
    sample_y2=np.array(sample_y2)

    #print(sample_x.shape)
    #print(sample_x1.shape)
    #print(sample_x2.shape)
    xtable= [[ 0 for i in range(100) ] for j in range(1500)]
    ytable= [[ 0 for i in range(100) ] for j in range(1500)]
    for i in range(100):
        for j in range(1500):
            x=[]
            x1=[]
            x2=[]
            x.append(sample_x[1500*i+j])
            x1.append(sample_x1[1500*i+j])
            x2.append(sample_x2[1500*i+j])
            xtable[j][i]=x+x1+x2
    for i in range(100):
        for j in range(1500):
            x=[]
            x1=[]
            x2=[]
            x.append(sample_y[1500*i+j])
            #x1.append(sample_y1[1500*i+j])
            #x2.append(sample_y2[1500*i+j])
            ytable[j][i]=x
    uid =str(num)+'-'+str(idx_start)+'-'+str(idx_end)
    return uid,xtable,ytable



In [88]:
uid,xtrain,ytrain=randomTrainingExample(dict_df)

In [65]:
#a = np.array(xtrain)
#a[0][0]
uid

'2-29101469-29251469'

In [89]:
xtrain=np.array(xtrain)
ytrain=np.array(ytrain)

In [90]:
xtrain= xtrain.astype(np.float32)
ytrain= ytrain.astype(np.float32)

In [91]:
xtrain

array([[[ 3.0000000e+00,  2.0000000e+00, -1.0000000e+00],
        [ 3.0000000e+00, -1.0000000e+00,  5.0000000e+00],
        [ 5.0000000e+00, -2.0000000e+00,  0.0000000e+00],
        ...,
        [ 3.0000000e+00, -4.0000000e+00,  9.0000000e+00],
        [ 2.0000000e+00, -6.0000000e+00,  1.0000000e+01],
        [ 8.0000000e+00,  2.0000000e+00, -3.0000000e+00]],

       [[ 5.0000000e+00,  1.0000000e+00, -2.0000000e+00],
        [ 2.0000000e+00,  4.0000000e+00, -2.0000000e+00],
        [ 3.0000000e+00, -2.0000000e+00,  3.0000000e+00],
        ...,
        [-1.0000000e+00,  5.0000000e+00, -4.0000000e+00],
        [-4.0000000e+00,  4.0000000e+00, -8.0000000e+00],
        [ 1.0000000e+01, -1.0000000e+00,  0.0000000e+00]],

       [[ 6.0000000e+00, -1.0000000e+00, -4.0000000e+00],
        [ 6.0000000e+00,  2.0000000e+00, -3.0000000e+00],
        [ 1.0000000e+00,  1.0000000e+00,  4.0000000e+00],
        ...,
        [ 4.0000000e+00,  1.0000000e+00,  1.0000000e+00],
        [ 0.0000000e+00, -4.0

In [102]:
num_epochs = 100
batch_size = 10
learning_rate = 1e-3

In [93]:
img_transform = transforms.Compose([
    transforms.ToTensor()
])


In [127]:
xtrainimg=img_transform(xtrain)
ytrainimg=img_transform(ytrain)

In [95]:
dict_final_x={}
dict_final_y={}

In [97]:
n_examples=17*10
now=time.time()
while(n_examples):
    while(uid in dict_final_x.keys()):
        uid,xtrain,ytrain=randomTrainingExample(dict_df)
    xtrain=np.array(xtrain)
    ytrain=np.array(ytrain)
    xtrain= xtrain.astype(np.float64)
    ytrain= ytrain.astype(np.float64)
    dict_final_x[uid]=xtrainimg
    dict_final_y[uid]=ytrainimg
    n_examples-=1
then=time.time()
print("Time Taken:",then-now)

Time Taken: 159.6667890548706


In [99]:
dict_final_y[uid].shape

torch.Size([1, 1500, 100])

In [146]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=3, padding=1),  # b, 16, 10, 10
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # b, 16, 5, 5
            nn.Conv2d(16, 8, 3, stride=2, padding=1),  # b, 8, 3, 3
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1)  # b, 8, 2, 2
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, 3, stride=2),  # b, 16, 5, 5
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 5, stride=3, padding=1),  # b, 8, 15, 15
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 1, 2, stride=2, padding=1),  # b, 1, 28, 28
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [147]:
model = autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,
weight_decay=1e-5)

In [139]:
#the custom dataloader
def get_x(uid_list):
    for i,uid in enumerate(uid_list):
        x=dict_final_x[uid]
        x=x.view(1,x.shape[0],x.shape[1],x.shape[2])
        if(i==0):
            out=x
        else:
            out = torch.cat((out, x), 0)
    return out
        
def get_y(uid_list):
    for i,uid in enumerate(uid_list):
        y=dict_final_y[uid]
        y=y.view(1,y.shape[0],y.shape[1],y.shape[2])
        if(i==0):
            out=y
        else:
            out = torch.cat((out, y), 0)
    return out
        

In [142]:
dataloader = DataLoader(list(dict_final_x.keys()), batch_size=batch_size, shuffle=True)

In [148]:
for epoch in range(num_epochs):
    for data in dataloader:
        img = get_x(data)
        img_out=get_y(data)
        img = Variable(img).cuda()
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img_out)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch+1, num_epochs, loss.data[0]))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
save_image(pic, './dc_img/image_{}.png'.format(epoch))

RuntimeError: The size of tensor a (1492) must match the size of tensor b (1500) at non-singleton dimension 2

In [144]:
a=0
for data in dataloader:
    print(get_y(data).shape)    

torch.Size([10, 1, 1500, 100])
torch.Size([10, 1, 1500, 100])
torch.Size([10, 1, 1500, 100])
torch.Size([10, 1, 1500, 100])
torch.Size([10, 1, 1500, 100])
torch.Size([10, 1, 1500, 100])
torch.Size([10, 1, 1500, 100])
torch.Size([10, 1, 1500, 100])
torch.Size([10, 1, 1500, 100])
torch.Size([10, 1, 1500, 100])
torch.Size([10, 1, 1500, 100])
torch.Size([10, 1, 1500, 100])
torch.Size([10, 1, 1500, 100])
torch.Size([10, 1, 1500, 100])
torch.Size([10, 1, 1500, 100])
torch.Size([10, 1, 1500, 100])
torch.Size([10, 1, 1500, 100])
torch.Size([10, 1, 1500, 100])
torch.Size([7, 1, 1500, 100])


In [133]:
xtrainimg.shape[1]

3

In [129]:
xtrainimg=xtrainimg.view(1,3,1500,100)

In [130]:
third_tensor = torch.cat((xtrainimg, xtrainimg), 0)

In [131]:
third_tensor.size()

torch.Size([2, 3, 1500, 100])